### This is the BackTest Platform for the strategy on the tradebot.

In [1]:
### This is the Beginning loadout of libraries for the backtest.
import pandas as pd  ### Pandas is a library used for data manipulation & datasets
import numpy as np   ### Numpy is a library used for sciencific computing, very useful and it is built from C++
import talib         ### Talib is a library that supports technical analysis and many functions for the stock trading.
import time
from datetime import datetime, timedelta  ### Datetime package for calculating time
from binance.client import Client         ### API to connect Binance
from binance.enums import *
from IPython.display import display, clear_output   ### A package that support displaying charts, datasets, and clearing output
import progressbar   ### A library to show a progressbar when a For Loop is happening.

In [2]:
### Binance Client Keys for login & connect to Binance Account
client=Client(api_key = "YOUR API KEY",
              api_secret = "YOUR SECRET KEY")

In [3]:
def sar(acceleration, maximum):    ### The function of SAR technical indicator
    low = []    ### create new lists of lowest & highest price to be used for SAR calculation by Talib
    high = []
    for i in range(len(back_test.close)):   ### Because Talib needs the data to be an array, so we have to put the highest
        low.append(back_test.low[i])        ### And Lowest data into the new lists created and made it into a dataframe so
        high.append(back_test.high[i])      ### it becomes an array.
    data = {"high":high,"low":low}   
    df = pd.DataFrame(data)
    tab_sar = talib.SAR(df['high'],df['low'],acceleration,maximum)
    
    list_sar = tab_sar.tolist()   ### The output from talib isn't a dataframe yet, so use .tolist() to make it into a dataframe
    return list_sar               

In [4]:
def kdj(kdj_length, k, d, ret):   ### The function of KDJ technical indicator
    RSV = []    ### create new lists to store values for calculation
    kvalue = []
    dvalue = []
    jvalue = []
    c = 1
                           ### Please checkout KDJ indicator formula.
    while c < k:        
        kvalue.append(50)    ### based on k and d period, we have to wait till the end of that period has reached 
        dvalue.append(50)    ### so we can start calculating the values, just like moving average.
        RSV.append(50)
        jvalue.append(3*kvalue[-1]-2*dvalue[-1])
        c = c + 1
    
    for j in range(k-1, len(back_test.close)):

        k_period_close = back_test.close[j-k+1:j+1]
        k_period_high = back_test.high[j-k+1:j+1]
        k_period_low = back_test.low[j-k+1:j+1]
 
        lowest = min(k_period_low)
        highest = max(k_period_high)
        if highest - lowest == 0:
            RSV.append(50)
        else:
            RSV.append(100*(back_test.close[j] - lowest)/(highest-lowest))
        kvalue.append(2/3*kvalue[j-1]+RSV[j]/3)
        dvalue.append(2/3*dvalue[j-1]+kvalue[j]/3)
        jvalue.append(3*kvalue[j]-2*dvalue[j]) 
    
    if ret == 'k':       ### based on the call in back_test function the KDJ fucntion will return different values
        return kvalue
    
    elif ret == 'd':
        return dvalue
    
    elif ret == 'j':
        return jvalue

In [5]:
def boll(boll_length, boll_multipler, boll_ret): ### The function of Bollinger Bands technical indicator
    result: List[Union[float, str]] = []         ### Create new lists to store values for caluclation
    upper: List[Union[float, str]] = []          ### Please chechout BOLL formula
    lower: List[Union[float, str]] = []
    
    for i in range(len(back_test.close)):        
        if i < boll_length:                      ### Similiar as KDJ, where we have to wait untill the end of the 
            result.append("-")
            upper.append("-")
            lower.append("-")
            continue
        sum_total = 0.0
        close = []

        for j in range(boll_length):
            close.append(float(back_test.close[i - j]))
            sum_total += float(back_test.close[i - j])
        result.append(abs(float("%.6f" % (sum_total / boll_length))))
        std = np.std(close)
        upper.append(result[-1] + std*2)
        lower.append(result[-1] - std*2)
        
    if boll_ret == "sma":    ### Same as KDJ where BOLL function will return various results based on calls from back_test function
        return result
    elif boll_ret == "upper":
        return upper
    elif boll_ret == "lower":
        return lower

In [6]:
### THE BACKTEST MAIN FUNCTION
def back_test(kdj_length, k, d, boll_length, boll_multipler, acceleration, maximum,
              start_at=None,end_at=None, symbol=None,cash=0,quantity=0, 
              profit1=1.5, profit2=2.0, profit3=2.8, profit4=3.8, stop_loss_force=0.015, stop_loss_pct=1.5):
    
    ### FIRST CREATE DICTIONARIES FOR EACH PERIOD, BUT I AM GOING TO DELETE MOST OF THEM LEAVING ONLY SOS 
    ### BECAUSE I FOUND EVERYTHING ARE USELESS FOR INSIGHTS EXCEPT SOS WHICH YOU BACKTEST FROM SPECIFIC PERIOD
   
    sos = {'Symbol':[], 'Total Realized Gain/Loss %':[], 'Average Realized Gain/Loss %':[], \
           'Positive Liquidation Average % Gain':[], 'Negative Liquidation Average % Loss':[], 'Positive Liquidation Probability':[]}
    
    ### This section calculates the days from today to previous date such as 1 year, month, or week ago.
    n = 1
    today = datetime.today()
    today = today.strftime("%Y/%m/%d, %H:%M")
    today_new = datetime.strptime(today, "%Y/%m/%d, %H:%M")
    dict_setup = {"time":[],"open":[], "high":[], "low":[], "close":[]}
    minute_data = []
            
    
    ### Because the only useful periods are the bullish and bearish periods, where monthly, weekly data aren't giving much info
    ### so I only use Start_at that starts and ends at the period you specified.
    if start_at is not None:
        end_at_datetime = datetime.strptime(end_at, "%Y/%m/%d, %H:%M")     ### Calculates the days from the start_at variable
        start_at_datetime = datetime.strptime(start_at, "%Y/%m/%d, %H:%M") ### to the end_at variable
        time_diff = end_at_datetime - start_at_datetime
        time_diff = time_diff.days
        while n <= time_diff-1:
            start_at_fixed = end_at_datetime - timedelta(days=time_diff-n)   ### convert the days into miliseconds that is
            start_at_mili = round(start_at_fixed.timestamp()*1000)           ### readable for binance api then make a call for data import
            klines = client.futures_klines(symbol = symbol, contractType = "PERPETUAL", 
                                           interval = Client.KLINE_INTERVAL_15MINUTE,startTime=start_at_mili, limit=96)
            minute_data.append(klines)   ### store all the imported data into a list
            n = n + 1                    ### n here is to count the days of data have been stored
            print("15 minutes kline data from "+start_at+" found! Extracted",n,"days of data!")  ### a message mentions the progress
            clear_output(wait=True)   ### clear the output when rerun
            time.sleep(0.07)          ### set a limit timer here to slow down data import a bit, otherwise 
                                      ### binance api will stop data import because it is importing too many data within 
                                      ### a short time
                    
        print("Completed one minute kline data extraction!") ### message delcaring the current situation
        print("")
        print("Currently creating kline dataframe!")
        time.sleep(1.0)    ### halt the function a bit here so that messages will show in proper order before 
                           ### creating a dataframe that has every data for the backtest
    

    p = progressbar.ProgressBar()   
    for m in p(range(len(minute_data))):
        for l in range(len(minute_data[m])):
            raw_date = minute_data[m][l][0]
            close = float(minute_data[m][l][4])
            op = float(minute_data[m][l][1])
            high = float(minute_data[m][l][2])      ### This part is where the data will be stored into previously created empty
            low = float(minute_data[m][l][3])       ### dictionary in order to form a dataframe
            date = datetime.fromtimestamp(int(raw_date/1000))
            readable_date = date.strftime("%Y/%m/%d, %H:%M")
            dict_setup["time"].append(readable_date)
            dict_setup['open'].append(op)
            dict_setup['high'].append(high)
            dict_setup['low'].append(low)
            dict_setup['close'].append(close)
    
    back_test.close = dict_setup['close']
    back_test.high = dict_setup['high']
    back_test.low = dict_setup['low']
    
    dict_setup['k'] = kdj(kdj_length, k, d, 'k')           ### calling kdj, boll & sar data from the respective functions
    dict_setup['d'] = kdj(kdj_length, k, d, 'd')
    dict_setup['j'] = kdj(kdj_length, k, d, 'j')
    dict_setup['boll upper'] = boll(boll_length, boll_multipler, 'upper')
    dict_setup['boll lower'] = boll(boll_length, boll_multipler, 'lower')
    dict_setup['boll mid'] = boll(boll_length, boll_multipler, 'sma')
    dict_setup['sar'] = sar(acceleration, maximum)
        
    df = pd.DataFrame(dict_setup)
    df.sort_values('time',inplace=True,ascending=True)    ### creates a dataframe and sort the order from start datetime to 
    df.set_index('time',inplace=True)                     ### end datetime
    print("")
    print("kline dataframe created!")
                
    date_l = []

    if start_at != None:
        date_l.append(start_at)                          ### Originally was created to calculate days for monthly, yearly
                                                         ### not needed anymore but still works.
    print("")
    print("Currently backtesting result!")
    time.sleep(1.0)
            
    for d in date_l:
        had = df     ### use the previously created dataframe and start from 20 because boll only begins to show data from 
        had_start = had.loc[had.head(20).index]  ### index 20
        had_calc = {"Time":[had_start.index[0]], "Type":["N/A"], "Price":[0], "Quantity":[0], "Buying Power":[cash],\
                    "Asset Value":[0], "Total Quantity":[0], "Realized Gain/Loss %":[0]}
        had_calc_df = pd.DataFrame(had_calc)   ### create another dataframe to store trade history, gain/loss results.

        down_l = []            ### These are the variables for various conditions during backtest
        x = 0
        buy_lock = 0
        stop_loss = 0
        profit_1 = 0
        profit_2 = 0
        profit_3 = 0
        profit_4 = 0
        profit_1_done = 0
        profit_2_done = 0
        profit_3_done = 0
        up_l = []
         
        ### THIS IS WHERE THE BACKTEST ACTUALLY BEGINS
        p = progressbar.ProgressBar()
        for a in p(range(20,len(had))): ### Start after 20 indices have passed and every technical indictors have the values.
            
            ### Starts storing kline, technical values into down_l list where it only stores candles that is negative and has
            ### lower closing price than the previous candle in down_l consecutively.
            ### This starts the backtest where since trade type is "N/A" so no trade has happened yet, so starts storing data
            ### for technical indicators signals
            if had.iloc[a,0] > had.iloc[a,3] and len(down_l)== 0 and \
            had_calc_df.iloc[-1,1] == 'N/A' and had.iloc[a,10] >= had.iloc[a,1]:
                down_l.append([had.iloc[a,0],had.iloc[a,1],had.iloc[a,2],had.iloc[a,3],had.iloc[a,4],had.iloc[a,5],\
                              had.iloc[a,6],had.iloc[a,7],had.iloc[a,8],had.iloc[a,9]])
                had_calc_df.iloc[-1,1] = 'Sell'
                up_l = []
            
            ### This part will avoid the situation where the kline data begins with positive candles because
            ### without negative candles going down for a period of time, this strategy can not figure out
            ### whether it is time to buy in, such as buy the dip
            if had.iloc[a,0] > had.iloc[a,3] and len(down_l) == 0 and len(up_l) != 0 and \
            had_calc_df.iloc[-1,1] == "Sell" and buy_lock == 1 and had.iloc[a,10] >= had.iloc[a,1]:
                buy_lock = 0
                up_l = []
                down_l.append([had.iloc[a,0],had.iloc[a,1],had.iloc[a,2],had.iloc[a,3],had.iloc[a,4],had.iloc[a,5],\
                              had.iloc[a,6],had.iloc[a,7],had.iloc[a,8],had.iloc[a,9]])
            
            ### This part will continue to store negative candles after the backtest started to simulate trade.
            if had.iloc[a,0] > had.iloc[a,3] and len(down_l) != 0 and had.iloc[a,10] >= had.iloc[a,1] and \
            (had_calc_df.iloc[-1,1] == 'Sell' or had_calc_df.iloc[-1,1] == 'N/A') and had.iloc[a,3] < down_l[-1][3]:
                down_l.append([had.iloc[a,0],had.iloc[a,1],had.iloc[a,2],had.iloc[a,3],had.iloc[a,4],had.iloc[a,5],\
                              had.iloc[a,6],had.iloc[a,7],had.iloc[a,8],had.iloc[a,9]])
            
            ### When the backtest ordered a buy, this part will find out at which position did backtest bought.
            if len(up_l) != 0 and (had_calc_df.iloc[-1,1] != 'Sell' and had_calc_df.iloc[-1,1] != 'N/A'):
                for buy_item in range(len(up_l)):
                    if 'Buy' in up_l[buy_item]:
                        buy_price = up_l[buy_item][3]
                        break
            
            ### this calcualtes the pass 20 candles overall amplitude to create an measurement of loss reduction when
            ### backtest just bought in and won't sell the holdings too early because of price going
            ### down a bit, unless an unusual price drop is happening.
            twenty_history_data = had.loc[had.index[a-20]:had.index[a]]
            twenty_history_sum_chg_pct = 0
            for t in range(len(twenty_history_data.index)):
                twenty_history_sum_chg_pct += np.absolute((twenty_history_data.iloc[t,1] - twenty_history_data.iloc[t,2])/
                                               twenty_history_data.iloc[t,2])
            avg_thcp = twenty_history_sum_chg_pct / len(twenty_history_data.index)
            
            ### This part will check which border of BOLL did the last 3 negative candles in down_l have touched
            ### so the backtest can buy in by looking where which border of BOLL did down_l candles crossed.
            if len(down_l) != 0:
                check_pass_mid1 = 0
                check_oversell1 = 0
                check_pass_upper1 = 0
                empty_true1 = 1
                if len(down_l) < 3:
                    for s1 in range(len(down_l)):
                        if down_l[s1][9] != '-' and down_l[s1][8] != '-' and down_l[s1][7] != '-':
                            if down_l[s1][3] < down_l[s1][8]:
                                check_oversell1 = 1
                                check_pass_mid1 = 1
                                check_pass_upper1 = 1
                                empty_true1 = 0
                            elif down_l[s1][3] < down_l[s1][9] and down_l[s1][3] >= down_l[s1][8]:
                                check_pass_mid1 = 1
                                check_pass_upper1 = 1
                                empty_true1 = 0
                            elif down_l[s1][3] < down_l[s1][7] and down_l[s1][3] >= down_l[s1][9]:
                                check_pass_upper1 = 1
                                empty_true1 = 0
                
                elif len(down_l) >= 3:
                    for s1 in range(len(down_l)-3,len(down_l)):
                        if down_l[s1][9] != '-' and down_l[s1][8] != '-' and down_l[s1][7] != '-':
                            if down_l[s1][3] < down_l[s1][8]:
                                check_oversell1 = 1
                                check_pass_mid1 = 1
                                check_pass_upper1 = 1
                                empty_true1 = 0
                            elif down_l[s1][3] < down_l[s1][9] and down_l[s1][3] >= down_l[s1][8]:
                                check_pass_mid1 = 1
                                check_pass_upper1 = 1
                                empty_true1 = 0
                            elif down_l[s1][3] < down_l[s1][7] and down_l[s1][3] >= down_l[s1][9]:
                                check_pass_upper1 = 1
                                empty_true1 = 0
            
            ### This is the buy in part where backtest will use the conditions set on 
            ### the technical indicators and various requirements to decide whether to buy or not.
            ### If buy in, this part will create an up_l list storing the positive candles higher than the previous one 
            ### consecutively for profit sell conditions.
            ### This part will also create trade data in had_calc_df dataframe for gain/loss calculation.
            if len(down_l) != 0 and had.iloc[a,0] < had.iloc[a,3] and buy_lock == 0 and had.iloc[a,2] >= had.iloc[a,10] and \
            (had.iloc[a,4] > had.iloc[a,5] or (had.iloc[a,3] - had.iloc[a,0])/had.iloc[a,0] > avg_thcp*1.5) and \
            had_calc_df.iloc[-1,1] == 'Sell':
                                
                if check_pass_mid1 == 0 and check_pass_upper1 == 1 and empty_true1 == 0:
                    
                    if had.iloc[a,4] > had.iloc[a,5]:
                        buy_in = had.iloc[a,3]
                    
                    if (had.iloc[a,3] - had.iloc[a,0])/had.iloc[a,0] > avg_thcp*1.5:
                        buy_in = had.iloc[a,0] + had.iloc[a,0]*avg_thcp*1.5
                    
                    
                    up_l.append([had.iloc[a,0],had.iloc[a,1],had.iloc[a,2],had.iloc[a,3],had.iloc[a,7],had.iloc[a,8],
                                had.iloc[a,9],'Buy']) 
                    x += 1
                    Time = had.index[a]
                    Type = "Buy above mid"
                    Price = buy_in + buy_in*0.00005
                    Quantity = quantity
                    Buying_Power = had_calc_df['Buying Power'][x-1] - Price*Quantity - Price*Quantity*0.0002
                    Asset_Value = had_calc_df['Total Quantity'][x-1]*Price + Price*Quantity
                    Total_Quantity = had_calc_df['Total Quantity'][x-1] + Quantity
                    Realized_Gain_Loss = 0
                    had_calc_df.loc[x] = [Time,Type,Price,Quantity,Buying_Power,Asset_Value,Total_Quantity,Realized_Gain_Loss]

                    down_l = []

                elif check_pass_mid1 == 1 and check_oversell1 == 0 and empty_true1 == 0:
                    
                    if had.iloc[a,4] > had.iloc[a,5]:
                        buy_in = had.iloc[a,3]
                    
                    if (had.iloc[a,3] - had.iloc[a,0])/had.iloc[a,0] > avg_thcp*1.5:
                        buy_in = had.iloc[a,0] + had.iloc[a,0]*avg_thcp*1.5
                    
                    up_l.append([had.iloc[a,0],had.iloc[a,1],had.iloc[a,2],had.iloc[a,3],had.iloc[a,7],had.iloc[a,8],
                                had.iloc[a,9],'Buy']) 

                    if had.iloc[a,3] > had.iloc[a,9]:
                        buy_type = 'Buy above mid'

                    else:
                        buy_type = "Buy pass mid"

                    x += 1
                    Time = had.index[a]
                    Type = buy_type
                    Price = buy_in + buy_in*0.00005
                    Quantity = quantity
                    Buying_Power = had_calc_df['Buying Power'][x-1] - Price*Quantity - Price*Quantity*0.0002
                    Asset_Value = had_calc_df['Total Quantity'][x-1]*Price + Price*Quantity
                    Total_Quantity = had_calc_df['Total Quantity'][x-1] + Quantity
                    Realized_Gain_Loss = 0
                    had_calc_df.loc[x] = [Time,Type,Price,Quantity,Buying_Power,Asset_Value,Total_Quantity,Realized_Gain_Loss]

                    down_l = []

                elif check_oversell1 == 1 and empty_true1 == 0:
                    
                    if had.iloc[a,4] > had.iloc[a,5]:
                        buy_in = had.iloc[a,3]
                    
                    if (had.iloc[a,3] - had.iloc[a,0])/had.iloc[a,0] > avg_thcp*1.5:
                        buy_in = had.iloc[a,0] + had.iloc[a,0]*avg_thcp*1.5
                    
                    up_l.append([had.iloc[a,0],had.iloc[a,1],had.iloc[a,2],had.iloc[a,3],had.iloc[a,7],had.iloc[a,8],
                                had.iloc[a,9],'Buy']) 

                    if had.iloc[a,3] > had.iloc[a,9]:
                        buy_type = 'Buy above mid'

                    elif had.iloc[a,3] <= had.iloc[a,9] and had.iloc[a,3] >= had.iloc[a,8]:
                        buy_type = "Buy pass mid"

                    elif had.iloc[a,3] < had.iloc[a,8]:
                        buy_type = "Buy oversell"

                    x += 1
                    Time = had.index[a]
                    Type = buy_type
                    Price = buy_in + buy_in*0.00005
                    Quantity = quantity
                    Buying_Power = had_calc_df['Buying Power'][x-1] - Price*Quantity - Price*Quantity*0.0002
                    Asset_Value = had_calc_df['Total Quantity'][x-1]*Price + Price*Quantity
                    Total_Quantity = had_calc_df['Total Quantity'][x-1] + Quantity
                    Realized_Gain_Loss = 0
                    had_calc_df.loc[x] = [Time,Type,Price,Quantity,Buying_Power,Asset_Value,Total_Quantity,Realized_Gain_Loss]

                    down_l = []
            
            
            ### If the above buy in conditions were not met, then if the candle is a positive candle this part
            ### will store the candle data into up_l list.
            elif had.iloc[a,0] < had.iloc[a,3] and had.iloc[a,2] >= had.iloc[a,10] and (buy_lock == 1 or \
            had.iloc[a,4] <= had.iloc[a,5]) and (had_calc_df.iloc[-1,1] == 'Sell' or had_calc_df.iloc[-1,1] == 'N/A'):
                if len(up_l) != 0:
                    if had.iloc[a,3] > up_l[-1][3]:
                        up_l.append([had.iloc[a,0],had.iloc[a,1],had.iloc[a,2],had.iloc[a,3],had.iloc[a,7],had.iloc[a,8],
                                    had.iloc[a,9]]) 

                elif len(up_l) == 0:
                    up_l.append([had.iloc[a,0],had.iloc[a,1],had.iloc[a,2],had.iloc[a,3],had.iloc[a,7],had.iloc[a,8],
                                had.iloc[a,9]])  
                
            
            ### If a buy in have occurred then this part will see whether or not the sell or profit sell conditions have met
            elif len(up_l) != 0 and had.iloc[a,2] > up_l[-1][3] and \
            (had_calc_df.iloc[-1,1] == 'Buy oversell' or had_calc_df.iloc[-1,1] == 'Buy pass mid' or \
            had_calc_df.iloc[-1,1] == 'Buy above mid' or had_calc_df.iloc[-1,1] == "Profit Sell"):
                up_l.append([had.iloc[a,0],had.iloc[a,1],had.iloc[a,2],had.iloc[a,3],had.iloc[a,7],had.iloc[a,8],
                                    had.iloc[a,9]]) 

            elif len(up_l) != 0 and had.iloc[a,2] < up_l[-1][3] and \
            (had_calc_df.iloc[-1,1] == 'Buy oversell' or had_calc_df.iloc[-1,1] == 'Buy pass mid' or \
            had_calc_df.iloc[-1,1] == 'Buy above mid' or had_calc_df.iloc[-1,1] == "Profit Sell"):
                
                ### this calculates the average % change of the canldes in up_l list for a general understanding of how
                ### much avg % u currently have gained and to secure profit before price dropping down
                sum_up_chg_pct = 0 
                for u in range(len(up_l)):
                    sum_up_chg_pct += (up_l[u][1] - up_l[u][2])/up_l[u][2]
                avg_up_chg_pct = sum_up_chg_pct / len(up_l)
                
                ### this part will setup values to secure profit when the current closing price have reached 
                ### specified limits
                if had.iloc[a,2] >= had.iloc[a,10] and (had.iloc[a,1] - buy_price)/buy_price >= avg_thcp*profit4 and \
                profit_4 == 0:
                    profit_4 = 1
                    final_price = buy_price + buy_price*avg_thcp*profit4
                
                elif had.iloc[a,2] >= had.iloc[a,10] and (had.iloc[a,1] - buy_price)/buy_price >= avg_thcp*profit3 and \
                profit_3 == 0:
                    profit_3 = 1
                    final_price = buy_price + buy_price*avg_thcp*profit3
                
                elif had.iloc[a,2] >= had.iloc[a,10] and (had.iloc[a,1] - buy_price)/buy_price >= avg_thcp*profit2 and \
                profit_2 == 0:
                    profit_2 = 1
                    final_price = buy_price + buy_price*avg_thcp*profit2
                
                elif had.iloc[a,2] >= had.iloc[a,10] and (had.iloc[a,1] - buy_price)/buy_price >= avg_thcp*profit1 and \
                profit_1 == 0:
                    profit_1 = 1
                    final_price = buy_price + buy_price*avg_thcp*profit1

                ### this part is the loss reduction
                ### it calculates and compares the price drop % to average amplitude % of pass 20 candles 
                ### if the price dropping too low, it will trigger the condition and sell immediately
                
                if (had.iloc[a,2] >= had.iloc[a,10] and (up_l[-1][3] - had.iloc[a,2])/up_l[-1][3] > avg_thcp*stop_loss_pct) or \
                had.iloc[a,10] >= had.iloc[a,1] or \
                (had.iloc[a,2] >= had.iloc[a,10] and (up_l[-1][3] - had.iloc[a,2])/up_l[-1][3] > stop_loss_force):
                    
                    stop_loss = 1
                    
                    if had.iloc[a,10] >= had.iloc[a,1]:
                        final_price_force_sell = had.iloc[a,3]
                    
                    elif had.iloc[a,2] >= had.iloc[a,10] and (up_l[-1][3] - had.iloc[a,2])/up_l[-1][3] > stop_loss_force:
                        final_price_force_sell = up_l[-1][3] - up_l[-1][3]*stop_loss_force
                       
                    elif had.iloc[a,2] >= had.iloc[a,10] and (up_l[-1][3] - had.iloc[a,2])/up_l[-1][3] > avg_thcp*stop_loss_pct:
                        final_price_force_sell = up_l[-1][3] - up_l[-1][3]*avg_thcp*stop_loss_pct
                
                ### this is the trade part where it will check the conditions to see whether to loss reduction or secure profit
                
                if profit_1 == 1 and profit_1_done == 0:
                    
                    if had.iloc[a,3] > had.iloc[a,0]:
                        if had.iloc[a,3] > up_l[-1][3]:
                            up_l.append([had.iloc[a,0],had.iloc[a,1],had.iloc[a,2],had.iloc[a,3],had.iloc[a,7],had.iloc[a,8],
                            had.iloc[a,9]])
                    
                    x += 1
                    Time = had.index[a]
                    Type = "Profit Sell"
                    Price = final_price
                    Quantity = quantity/4
                    Buying_Power = had_calc_df['Buying Power'][x-1] + Price*Quantity - Price*Quantity*0.0002
                    Asset_Value = had_calc_df['Total Quantity'][x-1]*Price - Price*Quantity
                    Total_Quantity = had_calc_df['Total Quantity'][x-1] - Quantity
                    Realized_Gain_Loss = ((had_calc_df['Total Quantity'][x-1]*Price \
                                            - had_calc_df['Asset Value'][x-1]) \
                                            / had_calc_df['Asset Value'][x-1])*100
                    had_calc_df.loc[x] = [Time,Type,Price,Quantity,Buying_Power,Asset_Value,Total_Quantity,Realized_Gain_Loss]
                    
                    
                    profit_1_done = 1

                if profit_2 == 1 and profit_2_done == 0:
                    
                    if had.iloc[a,3] > had.iloc[a,0]:
                        if had.iloc[a,3] > up_l[-1][3]:
                            up_l.append([had.iloc[a,0],had.iloc[a,1],had.iloc[a,2],had.iloc[a,3],had.iloc[a,7],had.iloc[a,8],
                            had.iloc[a,9]])
                    
                    final_quantity = quantity - had_calc_df.iloc[-1,6]
                    
                    if final_quantity == 0:
                        complete_quantity = quantity/2
                        profit_1 = 1
                        profit_1_done1 = 1
                        
                    elif final_quantity == quantity/4:
                        complete_quantity = quantity/4
                        
                    x += 1
                    Time = had.index[a]
                    Type = "Profit Sell"
                    Price = final_price
                    Quantity = complete_quantity
                    Buying_Power = had_calc_df['Buying Power'][x-1] + Price*Quantity - Price*Quantity*0.0002
                    Asset_Value = had_calc_df['Total Quantity'][x-1]*Price - Price*Quantity
                    Total_Quantity = had_calc_df['Total Quantity'][x-1] - Quantity
                    Realized_Gain_Loss = ((had_calc_df['Total Quantity'][x-1]*Price \
                                            - had_calc_df['Asset Value'][x-1]) \
                                            / had_calc_df['Asset Value'][x-1])*100
                    had_calc_df.loc[x] = [Time,Type,Price,Quantity,Buying_Power,Asset_Value,Total_Quantity,Realized_Gain_Loss]

                    
                    profit_2_done = 1
                    
                if profit_3 == 1 and profit_3_done == 0:
                    
                    if had.iloc[a,3] > had.iloc[a,0]:
                        if had.iloc[a,3] > up_l[-1][3]:
                            up_l.append([had.iloc[a,0],had.iloc[a,1],had.iloc[a,2],had.iloc[a,3],had.iloc[a,7],had.iloc[a,8],
                            had.iloc[a,9]])
                    
                    final_quantity = quantity - had_calc_df.iloc[-1,6]
                    
                    if final_quantity == 0:
                        complete_quantity = quantity - quantity/4
                        profit_1 = 1
                        profit_1_done = 1
                        profit_2 = 1
                        profit_2_done = 1
                        
                    elif final_quantity == quantity/4:
                        complete_quantity = quantity/2
                        profit_2 = 1
                        profit_2_done = 1
                        
                    elif final_quantity == quantity/2:
                        complete_quantity = quantity/4
                        
                    x += 1
                    Time = had.index[a]
                    Type = "Profit Sell"
                    Price = final_price
                    Quantity = complete_quantity
                    Buying_Power = had_calc_df['Buying Power'][x-1] + Price*Quantity - Price*Quantity*0.0002
                    Asset_Value = had_calc_df['Total Quantity'][x-1]*Price - Price*Quantity
                    Total_Quantity = had_calc_df['Total Quantity'][x-1] - Quantity
                    Realized_Gain_Loss = ((had_calc_df['Total Quantity'][x-1]*Price \
                                            - had_calc_df['Asset Value'][x-1]) \
                                            / had_calc_df['Asset Value'][x-1])*100
                    had_calc_df.loc[x] = [Time,Type,Price,Quantity,Buying_Power,Asset_Value,Total_Quantity,Realized_Gain_Loss]

                    
                    profit_3_done = 1

                
                if profit_4 == 1:
                    
                    if had.iloc[a,3] > had.iloc[a,0]:
                        if had.iloc[a,3] > up_l[-1][3]:
                            up_l.append([had.iloc[a,0],had.iloc[a,1],had.iloc[a,2],had.iloc[a,3],had.iloc[a,7],had.iloc[a,8],
                            had.iloc[a,9]])
                    
                    final_quantity = quantity - had_calc_df.iloc[-1,6]
                    
                    if final_quantity == 0:
                        complete_quantity = quantity
                        profit_1 = 1
                        profit_1_done = 1
                        profit_2 = 1
                        profit_2_done = 1
                        profit_3 = 1
                        profit_3_done = 1
                        
                    elif final_quantity == quantity/4:
                        complete_quantity = quantity - quantity/4
                        profit_2 = 1
                        profit_2_done = 1
                        profit_3 = 1
                        profit_3_done = 1
                        
                    elif final_quantity == quantity/2:
                        complete_quantity = quantity/2
                        profit_3 = 1
                        profit_3_done = 1
                        
                    elif final_quantity == quantity - quantity/4:
                        complete_quantity = quantity/4
                        
                    x += 1
                    Time = had.index[a]
                    Type = "Sell"
                    Price = final_price
                    Quantity = complete_quantity
                    Buying_Power = had_calc_df['Buying Power'][x-1] + Price*Quantity - Price*Quantity*0.0002
                    Asset_Value = had_calc_df['Total Quantity'][x-1]*Price - Price*Quantity
                    Total_Quantity = had_calc_df['Total Quantity'][x-1] - Quantity
                    Realized_Gain_Loss = ((had_calc_df['Total Quantity'][x-1]*Price \
                                            - had_calc_df['Asset Value'][x-1]) \
                                            / had_calc_df['Asset Value'][x-1])*100
                    had_calc_df.loc[x] = [Time,Type,Price,Quantity,Buying_Power,Asset_Value,Total_Quantity,Realized_Gain_Loss]
                    
                    profit_1 = 0
                    profit_2 = 0
                    profit_3 = 0 
                    profit_4 = 0
                    profit_1_done = 0
                    profit_2_done = 0
                    profit_3_done = 0

                    buy_lock = 1
                
                if stop_loss == 1 and had_calc_df.iloc[-1,1] != 'Sell':
                    
                    final_quantity = had_calc_df.iloc[-1,6]

                    x += 1
                    Time = had.index[a]
                    Type = "Sell"
                    Price = final_price_force_sell
                    Quantity = final_quantity
                    Buying_Power = had_calc_df['Buying Power'][x-1] + Price*Quantity - Price*Quantity*0.0002
                    Asset_Value = had_calc_df['Total Quantity'][x-1]*Price - Price*Quantity
                    Total_Quantity = had_calc_df['Total Quantity'][x-1] - Quantity
                    Realized_Gain_Loss = ((had_calc_df['Total Quantity'][x-1]*Price \
                                            - had_calc_df['Asset Value'][x-1]) \
                                            / had_calc_df['Asset Value'][x-1])*100
                    had_calc_df.loc[x] = [Time,Type,Price,Quantity,Buying_Power,Asset_Value,Total_Quantity,Realized_Gain_Loss]
                    
                    buy_lock = 1
                    
                    stop_loss = 0
                
                ### if nothing triggers the sell conditions then checks if the candle is positive and its closing price
                ### is higher than the last candle in up_l and add it into up_l if it is.
                elif had.iloc[a,0] < had.iloc[a,3] and stop_loss == 0:
                    if len(up_l) == 0:
                        up_l.append([had.iloc[a,0],had.iloc[a,1],had.iloc[a,2],had.iloc[a,3],had.iloc[a,7],had.iloc[a,8],
                            had.iloc[a,9]])
                    
                    elif len(up_l) != 0:
                        if had.iloc[a,3] > up_l[-1][3]:
                            up_l.append([had.iloc[a,0],had.iloc[a,1],had.iloc[a,2],had.iloc[a,3],had.iloc[a,7],had.iloc[a,8],
                                had.iloc[a,9]])  
            
###-----------------------------------------------------------------------------------------------------------------------
        
        ### this is the final part where it will calculate the overall gain/loss, average gain/loss, trades made, and 
        ### the position of BOLL that traded at.
        if d == start_at:
            print(start_at,"report")
            
            buy_upper_count = {'Type':['Buy upper'], 'Positive Liquidation Count':[],'Negative Liquidation Count':[]}
            buy_above_mid_count = {'Type':['Buy above mid'], 'Positive Liquidation Count':[],'Negative Liquidation Count':[]}
            buy_pass_mid_count = {'Type':['Buy pass mid'], 'Positive Liquidation Count':[],'Negative Liquidation Count':[]}
            buy_oversell_count = {'Type':['Buy oversell'], 'Positive Liquidation Count':[],'Negative Liquidation Count':[]}
            
            buy_upper_df = had_calc_df.loc[had_calc_df["Type"] == 'Buy upper']        
            buy_above_mid_df = had_calc_df.loc[had_calc_df["Type"] == 'Buy above mid']
            buy_pass_mid_df = had_calc_df.loc[had_calc_df["Type"] == 'Buy pass mid']
            buy_oversell_df = had_calc_df.loc[had_calc_df["Type"] == 'Buy oversell']
            
            upper_mean_l = []
            above_mid_mean_l = []
            pass_mid_mean_l = []
            oversell_mean_l = []
            
            upper_positive_count = 0
            upper_negative_count = 0
            above_mid_positive_count = 0
            above_mid_negative_count = 0
            pass_mid_positive_count = 0
            pass_mid_negative_count = 0
            oversell_positive_count = 0
            oversell_negative_count = 0
            display(had_calc_df)
            sos_tgl = (((had_calc_df.iloc[-1,4] + had_calc_df.iloc[-1,5])-cash)/cash)*100
            if len((had_calc_df.loc[had_calc_df['Realized Gain/Loss %'] != 0]).index) != 0:
                for i in buy_upper_df.index:
                    if i+1 in had_calc_df.index:
                        if had_calc_df.iloc[i+1,7] > 0:
                            upper_positive_count += 1
                            upper_mean_l.append(had_calc_df.iloc[i+1,7]) 
                            
                        elif had_calc_df.iloc[i+1,7] < 0:
                            upper_negative_count += 1 
                            upper_mean_l.append(had_calc_df.iloc[i+1,7]) 
                            
                for i2 in buy_above_mid_df.index:
                    if i2+1 in had_calc_df.index:
                        if had_calc_df.iloc[i2+1,7] > 0:
                            above_mid_positive_count += 1
                            above_mid_mean_l.append(had_calc_df.iloc[i2+1,7])
                            
                        elif had_calc_df.iloc[i2+1,7] < 0:
                            above_mid_negative_count += 1
                            above_mid_mean_l.append(had_calc_df.iloc[i2+1,7])
                            
                for i3 in buy_pass_mid_df.index:
                    if i3+1 in had_calc_df.index:
                        if had_calc_df.iloc[i3+1,7] > 0:
                            pass_mid_positive_count += 1
                            pass_mid_mean_l.append(had_calc_df.iloc[i3+1,7])
                            
                        elif had_calc_df.iloc[i3+1,7] < 0:
                            pass_mid_negative_count += 1
                            pass_mid_mean_l.append(had_calc_df.iloc[i3+1,7])
                            
                for i4 in buy_oversell_df.index:
                    if i4+1 in had_calc_df.index:
                        if had_calc_df.iloc[i4+1,7] > 0:
                            oversell_positive_count += 1
                            oversell_mean_l.append(had_calc_df.iloc[i4+1,7])
                            
                        elif had_calc_df.iloc[i4+1,7] < 0:
                            oversell_negative_count += 1
                            oversell_mean_l.append(had_calc_df.iloc[i4+1,7])
                            
                sos_agl = sos_tgl/len((had_calc_df.loc[had_calc_df['Realized Gain/Loss %'] != 0]).index)
                sos_pl = (len((had_calc_df.loc[had_calc_df['Realized Gain/Loss %'] > 0]).index) \
                          /len((had_calc_df.loc[had_calc_df['Realized Gain/Loss %'] != 0]).index))*100
                sos_plag = (had_calc_df.loc[had_calc_df['Realized Gain/Loss %'] > 0])['Realized Gain/Loss %'].mean()
                sos_nlal = (had_calc_df.loc[had_calc_df['Realized Gain/Loss %'] < 0])['Realized Gain/Loss %'].mean()
            else:
                sos_agl = 0
                sos_pl = 0
            
            if len(above_mid_mean_l) > 0:
                buy_above_mid_mean = sum(above_mid_mean_l)/len(above_mid_mean_l)
            else:
                buy_above_mid_mean = 0
            buy_above_mid_count['Positive Liquidation Count'].append(above_mid_positive_count)
            buy_above_mid_count['Negative Liquidation Count'].append(above_mid_negative_count)
            above_mid_count_df = pd.DataFrame(buy_above_mid_count)
            
            if len(pass_mid_mean_l) > 0:
                buy_pass_mid_mean = sum(pass_mid_mean_l)/len(pass_mid_mean_l)
            else:
                buy_pass_mid_mean = 0
            buy_pass_mid_count['Positive Liquidation Count'].append(pass_mid_positive_count)
            buy_pass_mid_count['Negative Liquidation Count'].append(pass_mid_negative_count)
            pass_mid_count_df = pd.DataFrame(buy_pass_mid_count)
            
            if len(oversell_mean_l) > 0:
                buy_oversell_mean = sum(oversell_mean_l)/len(oversell_mean_l)
            else:
                buy_oversell_mean = 0
            buy_oversell_count['Positive Liquidation Count'].append(oversell_positive_count)
            buy_oversell_count['Negative Liquidation Count'].append(oversell_negative_count)
            oversell_count_df = pd.DataFrame(buy_oversell_count)

            sos['Symbol'].append(symbol)
            sos['Total Realized Gain/Loss %'].append(sos_tgl)
            sos['Average Realized Gain/Loss %'].append(sos_agl)
            sos['Positive Liquidation Average % Gain'].append(sos_plag)
            sos['Negative Liquidation Average % Loss'].append(sos_nlal)
            sos['Positive Liquidation Probability'].append(sos_pl)
            sos_df = pd.DataFrame(sos)
            sos_buy_count_df = had_calc_df[['Type','Price']].groupby('Type').count()
            sos_buy_count_df = sos_buy_count_df.rename(columns={"Price":"Number of Trade"})
    
    
    ### Ignore this part, it will be deleted.
    if start_at is None:
        for t in date_l:
            if t == onehour:
                print("")
                print("Hour")       
                print("")
                display(hoh_df)
                print("")
                print("The Number of Purchases:",hoh_buy_count_df)
                print("")
                print("--------------------------------------------------------------------------------------------------")
            if t == halfday:
                print("")
                print("Semi-Day")
                print("")
                display(hdohd_df)
                print("")
                print("The Number of Purchases:",hdohd_buy_count_df)
                print("")
                print("--------------------------------------------------------------------------------------------------")
            if t == oneday:
                print("")
                print("Day")
                print("")
                display(dod_df)
                print("")
                print("The Number of Purchases:",dod_buy_count_df)
                print("")
                print("--------------------------------------------------------------------------------------------------")
            if t == oneweek:
                print("")
                print("Week")
                print("")
                display(wow_df)
                print("")
                print("The Number of Purchases:",wow_buy_count_df)
                print("")
                print("--------------------------------------------------------------------------------------------------")
            if t == onemonth:
                print("")
                print("Month")
                print("")
                display(mom_df)
                print("")
                print("The Number of Purchases:",mom_buy_count_df)
                print("")
                print("--------------------------------------------------------------------------------------------------")
            if t == onequarter:
                print("")
                print("Quarter")
                print("")
                display(qoq_df)
                print("")
                print("The Number of Purchases:",qoq_buy_count_df)
                print("")
                print("--------------------------------------------------------------------------------------------------")
            if t == oneyear:
                print("")
                print("Annual")
                print("")
                display(yoy)
                print("")
                print("The Number of Purchases:",yoy_buy_count_df)
                print("")
                print("--------------------------------------------------------------------------------------------------")
    
    ### this part will display all the dataframe of the results.
    if start_at is not None:
        print("")
        print("--------------------------------------------------------------------------------------------------")
        print("")
        print(start_at+"report")
        print("")
        display(sos_df)
        print("")
        print("Lowest cash", min((had_calc_df['Buying Power']+had_calc_df['Asset Value'])))
        print("")
        print("Buy above mid stats")
        print("Above Mid Mean:",buy_above_mid_mean)
        display(above_mid_count_df)
        print("")
        print("Buy pass mid stats")
        print("Pass Mid Mean:",buy_pass_mid_mean)
        display(pass_mid_count_df) 
        print("")
        print("Buy oversell stats")
        print("Oversell Mean:",buy_oversell_mean)
        display(oversell_count_df) 
        print("")                
        print("The Number of Trades:")
        display(sos_buy_count_df)
        print("")
        print("--------------------------------------------------------------------------------------------------")

In [11]:
back_test(kdj_length = 3, k = 9, d = 3, boll_length = 20, boll_multipler = 2, acceleration = 0.02, maximum = 0.2,
        start_at = "2021/08/25, 20:00", end_at = "2021/08/28, 20:00",symbol='DOGEUSDT',cash=10000,quantity=26119,
         profit1=2.5, profit2=3.0, profit3=3.0, profit4=4.5, stop_loss_force=0.03, stop_loss_pct=3.5)

Completed one minute kline data extraction!

Currently creating kline dataframe!


100% (2 of 2) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00



kline dataframe created!

Currently backtesting result!


100% (87 of 87) |########################| Elapsed Time: 0:00:00 Time:  0:00:00


2021/08/25, 20:00 report


...


Buy oversell stats
Oversell Mean: 0


,Type,Positive Liquidation Count,Negative Liquidation Count
0,Buy oversell,0,0



The Number of Trades:


,Number of Trade
Type,
Buy above mid,3
Profit Sell,3
Sell,4



--------------------------------------------------------------------------------------------------


In [42]:
back_test(kdj_length = 3, k = 9, d = 3, boll_length = 20, boll_multipler = 2, acceleration = 0.02, maximum = 0.2,
        start_at = "2021/03/07, 20:00", end_at = "2021/03/10, 20:00",symbol='SKLUSDT',cash=1500,quantity=2000,
         profit1=2.5, profit2=3.0, profit3=4.8, profit4=5.5, stop_loss_force=0.02, stop_loss_pct=1.5)

Completed one minute kline data extraction!

Currently creating kline dataframe!


100% (2 of 2) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00



kline dataframe created!

Currently backtesting result!


100% (172 of 172) |######################| Elapsed Time: 0:00:00 Time:  0:00:00


2021/03/07, 20:00 report


...


Buy oversell stats
Oversell Mean: 0


,Type,Positive Liquidation Count,Negative Liquidation Count
0,Buy oversell,0,0



The Number of Trades:


,Number of Trade
Type,
Buy above mid,5
Buy pass mid,1
Profit Sell,4
Sell,7



--------------------------------------------------------------------------------------------------


In [153]:
back_test(kdj_length = 3, k = 9, d = 3, boll_length = 20, boll_multipler = 2, acceleration = 0.02, maximum = 0.2,
        start_at = "2020/08/09, 20:00", end_at = "2021/08/10, 20:00",symbol='ETHUSDT',cash=500000,quantity=8,
         profit1=1.5, profit2=2.0, profit3=2.8, profit4=3.8, stop_loss_force=0.015, stop_loss_pct=1.5)

Completed one minute kline data extraction!

Currently creating kline dataframe!


100% (365 of 365) |######################| Elapsed Time: 0:00:00 Time:  0:00:00



kline dataframe created!

Currently backtesting result!


 67% (23706 of 35020) |############      | Elapsed Time: 0:01:34 ETA:   0:00:44D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:460: RuntimeWarning: invalid value encountered in double_scalars
100% (35020 of 35020) |##################| Elapsed Time: 0:02:19 Time:  0:02:19


2020/08/09, 20:00 report


...


Buy oversell stats
Oversell Mean: 0


,Type,Positive Liquidation Count,Negative Liquidation Count
0,Buy oversell,0,0



The Number of Trades:


,Number of Trade
Type,
Buy above mid,902
Buy pass mid,403
Profit Sell,402
Sell,1306



--------------------------------------------------------------------------------------------------


In [154]:
back_test(kdj_length = 3, k = 9, d = 3, boll_length = 20, boll_multipler = 2, acceleration = 0.02, maximum = 0.2,
        start_at = "2020/08/09, 20:00", end_at = "2021/08/10, 20:00",symbol='BNBUSDT',cash=500000,quantity=800,
         profit1=1.5, profit2=2.0, profit3=2.8, profit4=3.8, stop_loss_force=0.015, stop_loss_pct=1.5)

Completed one minute kline data extraction!

Currently creating kline dataframe!


100% (365 of 365) |######################| Elapsed Time: 0:00:00 Time:  0:00:00



kline dataframe created!

Currently backtesting result!


 13% (4860 of 35020) |##                 | Elapsed Time: 0:00:19 ETA:   0:02:26D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:420: RuntimeWarning: invalid value encountered in double_scalars
 19% (6895 of 35020) |###                | Elapsed Time: 0:00:29 ETA:   0:01:47D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:537: RuntimeWarning: invalid value encountered in double_scalars
 59% (20983 of 35020) |##########        | Elapsed Time: 0:01:30 ETA:   0:01:09D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:460: RuntimeWarning: invalid value encountered in double_scalars
100% (35020 of 35020) |##################| Elapsed Time: 0:02:33 Time:  0:02:33


2020/08/09, 20:00 report


...


Buy oversell stats
Oversell Mean: 0


,Type,Positive Liquidation Count,Negative Liquidation Count
0,Buy oversell,0,0



The Number of Trades:


,Number of Trade
Type,
Buy above mid,927
Buy pass mid,402
Profit Sell,419
Sell,1330



--------------------------------------------------------------------------------------------------


In [12]:
back_test(kdj_length = 3, k = 9, d = 3, boll_length = 20, boll_multipler = 2, acceleration = 0.02, maximum = 0.2,
        start_at = "2020/09/16, 20:00", end_at = "2021/08/10, 20:00",symbol='BLZUSDT',cash=1500,quantity=2000,
         profit1=1.5, profit2=2.0, profit3=2.8, profit4=3.8, stop_loss_force=0.015, stop_loss_pct=1.5)

Completed one minute kline data extraction!

Currently creating kline dataframe!


100% (327 of 327) |######################| Elapsed Time: 0:00:00 Time:  0:00:00



kline dataframe created!

Currently backtesting result!


 78% (24564 of 31372) |##############    | Elapsed Time: 0:01:32 ETA:   0:00:25D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:537: RuntimeWarning: invalid value encountered in double_scalars
100% (31372 of 31372) |##################| Elapsed Time: 0:01:57 Time:  0:01:57


2020/09/16, 20:00 report


...


Buy oversell stats
Oversell Mean: 0


,Type,Positive Liquidation Count,Negative Liquidation Count
0,Buy oversell,0,0



The Number of Trades:


,Number of Trade
Type,
Buy above mid,837
Buy pass mid,362
Profit Sell,266
Sell,1200



--------------------------------------------------------------------------------------------------
